<a href="https://colab.research.google.com/github/awhitehouse1/project_voting/blob/main/cleaning_votes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Loading Data

In [101]:
import pandas as pd
import numpy as np

vdf = pd.read_csv('./data/voting_VA.csv') # Load historical voting data
cdf = pd.read_csv('./data/county_adjacencies.csv') # Load county adjacency data and population data
df = pd.read_csv('./data/0002_ds249_20205_county_E.csv', encoding='latin1')


<ipython-input-101-2fce079a44cf>:6: DtypeWarning: Columns (3,4,6,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266

## voting_VA dataset exploration and cleaning

In [13]:
vdf


,Unnamed: 0,year,state,state_po,county_name,county_fips,office,candidate,party,candidatevotes,totalvotes,version,mode
0,11161,2000,VIRGINIA,VA,ACCOMACK,51001,US PRESIDENT,AL GORE,DEMOCRAT,5092,11925,20220315,TOTAL
1,11162,2000,VIRGINIA,VA,ACCOMACK,51001,US PRESIDENT,GEORGE W. BUSH,REPUBLICAN,6352,11925,20220315,TOTAL
2,11163,2000,VIRGINIA,VA,ACCOMACK,51001,US PRESIDENT,RALPH NADER,GREEN,220,11925,20220315,TOTAL
3,11164,2000,VIRGINIA,VA,ACCOMACK,51001,US PRESIDENT,OTHER,OTHER,261,11925,20220315,TOTAL
4,11165,2000,VIRGINIA,VA,ALBEMARLE,51003,US PRESIDENT,AL GORE,DEMOCRAT,16255,36846,20220315,TOTAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3731,71966,2020,VIRGINIA,VA,WINCHESTER CITY,51840,US PRESIDENT,OTHER,OTHER,24,12113,20220315,ELECTION DAY
3732,71967,2020,VIRGINIA,VA,WINCHESTER CITY,51840,US PRESIDENT,OTHER,OTHER,0,12113,20220315,PROVISIONAL
3733,71968,2020,VIRGINIA,VA,WINCHESTER CITY,51840,US PRESIDENT,DONALD J TRUMP,REPUBLICAN,2825,12113,20220315,ABSENTEE
3734,71969,2020,VIRGINIA,VA,WINCHESTER CITY,51840,US PRESIDENT,DONALD J TRUMP,REPUBLICAN,2370,12113,20220315,ELECTION DAY


In [14]:
print(vdf.shape)
print(vdf.dtypes)

# All of the variables have the correct data type, so they do not need to be reassigned

(3736, 13)
Unnamed: 0         int64
year               int64
state             object
state_po          object
county_name       object
county_fips        int64
office            object
candidate         object
party             object
candidatevotes     int64
totalvotes         int64
version            int64
mode              object
dtype: object


In [15]:
print(vdf['party'].value_counts()) # Get the different party values and the number of rows for each party

print(vdf.groupby(["party"]).candidatevotes.sum())
# Non democrat and republican votes make up only 2.4% of all votes in the dataset

party
DEMOCRAT       1068
REPUBLICAN     1068
OTHER          1068
LIBERTARIAN     398
GREEN           134
Name: count, dtype: int64
party
DEMOCRAT       10997557
GREEN             59373
LIBERTARIAN       64761
OTHER            406335
REPUBLICAN     10433236
Name: candidatevotes, dtype: int64


Votes for third parties (libertarian, green, and other) make up only about 2.4% of all votes in the dataset. Because votes for the republican and democrat parties make up the majority of votes and the United States is dominated by a two party system, all third party votes will be dropped.

In [16]:
# Drop the third party votes
clean_vdf = vdf.drop(vdf[vdf['party'] == 'OTHER'].index)
clean_vdf = clean_vdf.drop(clean_vdf[clean_vdf['party'] == 'LIBERTARIAN'].index)
clean_vdf = clean_vdf.drop(clean_vdf[clean_vdf['party'] == 'GREEN'].index)
clean_vdf

,Unnamed: 0,year,state,state_po,county_name,county_fips,office,candidate,party,candidatevotes,totalvotes,version,mode
0,11161,2000,VIRGINIA,VA,ACCOMACK,51001,US PRESIDENT,AL GORE,DEMOCRAT,5092,11925,20220315,TOTAL
1,11162,2000,VIRGINIA,VA,ACCOMACK,51001,US PRESIDENT,GEORGE W. BUSH,REPUBLICAN,6352,11925,20220315,TOTAL
4,11165,2000,VIRGINIA,VA,ALBEMARLE,51003,US PRESIDENT,AL GORE,DEMOCRAT,16255,36846,20220315,TOTAL
5,11166,2000,VIRGINIA,VA,ALBEMARLE,51003,US PRESIDENT,GEORGE W. BUSH,REPUBLICAN,18291,36846,20220315,TOTAL
8,11169,2000,VIRGINIA,VA,ALLEGHANY,51005,US PRESIDENT,AL GORE,DEMOCRAT,2214,5123,20220315,TOTAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3725,71960,2020,VIRGINIA,VA,WINCHESTER CITY,51840,US PRESIDENT,JOSEPH R BIDEN JR,DEMOCRAT,1466,12113,20220315,ELECTION DAY
3726,71961,2020,VIRGINIA,VA,WINCHESTER CITY,51840,US PRESIDENT,JOSEPH R BIDEN JR,DEMOCRAT,51,12113,20220315,PROVISIONAL
3733,71968,2020,VIRGINIA,VA,WINCHESTER CITY,51840,US PRESIDENT,DONALD J TRUMP,REPUBLICAN,2825,12113,20220315,ABSENTEE
3734,71969,2020,VIRGINIA,VA,WINCHESTER CITY,51840,US PRESIDENT,DONALD J TRUMP,REPUBLICAN,2370,12113,20220315,ELECTION DAY


The county names in this dataset are not uniform with the county names in the county_adjacencies dataset. In order to make the county names equal, I removed the spaces from the names.

In [18]:
print(clean_vdf['county_name'])
clean_vdf['county_name'] = clean_vdf['county_name'].str.replace(" ", "")
print(clean_vdf['county_name'])

0             ACCOMACK
1             ACCOMACK
4            ALBEMARLE
5            ALBEMARLE
8            ALLEGHANY
             ...      
3725    WINCHESTERCITY
3726    WINCHESTERCITY
3733    WINCHESTERCITY
3734    WINCHESTERCITY
3735    WINCHESTERCITY
Name: county_name, Length: 2136, dtype: object


## county_adjacencies dataset exploration and cleaning

In [19]:
cdf

,County,Population2022,FIPS,District,N1,N2,N3,N4,N5,N6,N7,N8,N9,N10,N11,N12
0,Accomack,33191,51001,2,Northumberland,Lancaster,Middlesex,Mathews,Northampton,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Albemarle,114534,51003,5,CharlottesvilleCity,Nelson,Augusta,Rockingham,Greene,Orange,Louisa,Fluvanna,Buckingham,Nelson,NaN,NaN
2,AlexandriaCity,155525,51510,8,Fairfax,Arlington,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Alleghany,14835,51005,6,Craig,Bath,Botetourt,CovingtonCity,Rockbridge,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Amelia,13455,51007,5,PrinceEdward,Cumberland,Powhatan,Chesterfield,Dinwiddie,Nottoway,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,WinchesterCity,27936,51840,6,Frederick,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
129,Wise,35421,51195,9,Lee,Scott,Dickenson,NortonCity,Russell,NaN,NaN,NaN,NaN,NaN,NaN,NaN
130,Wythe,28111,51197,9,Bland,Smyth,Grayson,Carroll,Pulaski,NaN,NaN,NaN,NaN,NaN,NaN,NaN
131,York,71341,51199,1,Gloucester,JamesCty,WilliamsburgCity,NewportNewsCity,PoquosonCity,HamptonCity,Northampton,NaN,NaN,NaN,NaN,NaN


In [22]:
cdf.shape
cdf.dtypes
# The variables have the correct data types

County            object
Population2022     int64
FIPS               int64
District           int64
N1                object
N2                object
N3                object
N4                object
N5                object
N6                object
N7                object
N8                object
N9                object
N10               object
N11               object
N12               object
dtype: object

To make the county names for this dataset equal to the county names in the cleaned voting_VA dataset, the all of the letters need to be made uppercase.

In [24]:
clean_cdf = cdf
clean_cdf['County'] = cdf['County'].str.upper()
print(clean_cdf['County'])

0              ACCOMACK
1             ALBEMARLE
2        ALEXANDRIACITY
3             ALLEGHANY
4                AMELIA
             ...       
128      WINCHESTERCITY
129                WISE
130               WYTHE
131                YORK
132    WILLIAMSBURGCITY
Name: County, Length: 133, dtype: object


## NHGIS County data for 2016 to 2020

I chose to use this dataset since 2024 might be a Biden-Trump rematch, so data from these years may be relevant for predicting the upcoming election.

In [102]:
df

,GISJOIN,YEAR,STUSAB,REGIONA,DIVISIONA,STATE,STATEA,COUNTY,COUNTYA,COUSUBA,...,AMWSE004,AMWSE005,AMWSE006,AMWSE007,AMWSE008,AMWSE009,AMWSE010,AMWSE011,AMWSE012,AMWSE013
0,GIS Join Match Code,Data File Year,State Postal Abbreviation,Region Code,Division Code,State Name,State Code,County Name,County Code,County Subdivision Code,...,With an Internet subscription: Broadband of an...,With an Internet subscription: Cellular data plan,With an Internet subscription: Cellular data p...,With an Internet subscription: Broadband such ...,With an Internet subscription: Broadband such ...,With an Internet subscription: Satellite Inter...,With an Internet subscription: Satellite Inter...,With an Internet subscription: Other service w...,Internet access without a subscription,No Internet access
1,G0100010,2016-2020,AL,NaN,NaN,Alabama,01,Autauga County,001,NaN,...,17831,15390,2666,13780,2004,1695,349,57,402,3307
2,G0100030,2016-2020,AL,NaN,NaN,Alabama,01,Baldwin County,003,NaN,...,71498,65644,13937,53139,4634,8874,788,0,2149,10018
3,G0100050,2016-2020,AL,NaN,NaN,Alabama,01,Barbour County,005,NaN,...,6025,5351,1800,3620,547,933,79,0,563,2700
4,G0100070,2016-2020,AL,NaN,NaN,Alabama,01,Bibb County,007,NaN,...,5526,5074,1889,2669,307,1182,142,0,210,1520
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3217,G7201450,2016-2020,PR,NaN,NaN,Puerto Rico,72,Vega Baja Municipio,145,NaN,...,9225,8620,2767,5726,437,1572,119,17,277,9794
3218,G7201470,2016-2020,PR,NaN,NaN,Puerto Rico,72,Vieques Municipio,147,NaN,...,1640,1544,1102,411,74,189,0,0,16,738
3219,G7201490,2016-2020,PR,NaN,NaN,Puerto Rico,72,Villalba Municipio,149,NaN,...,5712,5483,2796,2535,182,347,26,0,190,1987
3220,G7201510,2016-2020,PR,NaN,NaN,Puerto Rico,72,Yabucoa Municipio,151,NaN,...,7734,7248,4088,2255,303,1588,143,0,51,3865


In [111]:
print(df.shape) # this dataset has 992 columns, which made it challenging to figure out which columns to use
print(df.dtypes)
# numeric values are stored as objects when they should be ints, will have to convert these to ints

(3221, 992)
GISJOIN      object
YEAR         object
STUSAB       object
REGIONA      object
DIVISIONA    object
              ...  
AMWSE009     object
AMWSE010     object
AMWSE011     object
AMWSE012     object
AMWSE013     object
Length: 992, dtype: object


In [108]:
df_clean = pd.DataFrame()
df = df.drop(df.index[0]) # remove the column names


df_clean['state'] = df['STATE']

# Make the county names equal to the other datasets
df_clean['county'] = df['COUNTY'].str.replace(" ", "")
df_clean['county'] = df_clean['county'].str.upper()



In [109]:
f_age_by_sex = ['AMPKE031', 'AMPKE032', 'AMPKE033', 'AMPKE034', 'AMPKE035', 'AMPKE036', 'AMPKE037', 'AMPKE038', 'AMPKE039', 'AMPKE040', 'AMPKE041', 'AMPKE042', 'AMPKE043', 'AMPKE044', 'AMPKE045','AMPKE046', 'AMPKE047', 'AMPKE048', 'AMPKE049']
m_age_by_sex = ['AMPKE007','AMPKE008', 'AMPKE009', 'AMPKE010', 'AMPKE011','AMPKE012', 'AMPKE013', 'AMPKE014', 'AMPKE015','AMPKE015', 'AMPKE016', 'AMPKE017', 'AMPKE018', 'AMPKE019', 'AMPKE020', 'AMPKE021', 'AMPKE022', 'AMPKE023', 'AMPKE024']


# convert the object values into ints
for col in m_age_by_sex:
  df[col] = pd.to_numeric(df[col], errors='coerce')

for col in f_age_by_sex:
  df[col] = pd.to_numeric(df[col], errors='coerce')

# combine age counts into larger groups
df_clean['M_18_to_29'] = df['AMPKE007'] + df['AMPKE008'] + df['AMPKE009'] + df['AMPKE010'] + df['AMPKE011']
df_clean['M_30_to_49'] = df['AMPKE012'] + df['AMPKE013'] + df['AMPKE014'] + df['AMPKE015']
df_clean['M_50_to_64'] = df['AMPKE016'] + df['AMPKE017'] + df['AMPKE018'] + df['AMPKE019']
df_clean['M_65_plus'] = df['AMPKE020'] + df['AMPKE021'] + df['AMPKE022'] + df['AMPKE023'] + df['AMPKE024']

df_clean['F_18_to_29'] = df['AMPKE031'] + df['AMPKE032'] + df['AMPKE033'] + df['AMPKE034'] + df['AMPKE035']
df_clean['F_30_to_49'] = df['AMPKE036'] + df['AMPKE037'] + df['AMPKE038'] + df['AMPKE039']
df_clean['F_50_to_64'] = df['AMPKE040'] + df['AMPKE041'] + df['AMPKE042'] + df['AMPKE043']
df_clean['F_65_plus'] = df['AMPKE044'] + df['AMPKE045'] + df['AMPKE046'] + df['AMPKE047'] + df['AMPKE048'] + df['AMPKE049']


In [110]:
income = ['AMR7E001', 'AMR7E002', 'AMR7E003', 'AMR7E004', 'AMR7E005', 'AMR7E006', 'AMR7E007', 'AMR7E008', 'AMR7E009', 'AMR7E010', 'AMR7E011', 'AMR7E012', 'AMR7E013', 'AMR7E014', 'AMR7E015', 'AMR7E016', 'AMR7E017']

# convert the object values into ints
for col in income:
  df[col] = pd.to_numeric(df[col], errors='coerce')

# combine income counts into larger groups
df_clean['income_under_30000'] = df['AMR7E002'] + df['AMR7E003'] + df['AMR7E004'] + df['AMR7E005'] + df['AMR7E006']
df_clean['income_30000_to_49999'] = df['AMR7E007'] + df['AMR7E008'] + df['AMR7E009'] + df['AMR7E009']
df_clean['income_50000_to_99999'] = df['AMR7E011'] + df['AMR7E012'] + df['AMR7E013']
df_clean['income_100000_plus'] = df['AMR7E014'] + df['AMR7E015'] + df['AMR7E016'] + df['AMR7E017']

df_clean

,state,county,M_18_to_29,M_30_to_49,M_50_to_64,M_65_plus,F_18_to_29,F_30_to_49,F_50_to_64,F_65_plus,income_under_30000,income_30000_to_49999,income_50000_to_99999,income_100000_plus
1,Alabama,AUTAUGACOUNTY,4089.0,7121.0,5394.0,3358.0,4206.0,7607.0,5589.0,4807.0,6089.0,3753.0,6553.0,5461.0
2,Alabama,BALDWINCOUNTY,14020.0,25506.0,21889.0,19060.0,13997.0,27145.0,24023.0,24115.0,18734.0,16867.0,25654.0,23451.0
3,Alabama,BARBOURCOUNTY,2311.0,3740.0,2443.0,1931.0,1542.0,2635.0,2356.0,2704.0,4147.0,1810.0,2203.0,1296.0
4,Alabama,BIBBCOUNTY,2052.0,3484.0,2360.0,1513.0,1518.0,2694.0,2006.0,2059.0,2452.0,1144.0,2535.0,1173.0
5,Alabama,BLOUNTCOUNTY,4257.0,7069.0,5843.0,4228.0,3895.0,7215.0,5722.0,5747.0,6372.0,4650.0,6073.0,4245.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3217,Puerto Rico,VEGABAJAMUNICIPIO,4160.0,5964.0,4716.0,4026.0,4047.0,6571.0,5574.0,5742.0,12510.0,4001.0,2752.0,480.0
3218,Puerto Rico,VIEQUESMUNICIPIO,557.0,1141.0,872.0,724.0,561.0,915.0,954.0,991.0,1849.0,565.0,89.0,34.0
3219,Puerto Rico,VILLALBAMUNICIPIO,1906.0,2383.0,2194.0,1659.0,1869.0,2649.0,2460.0,2127.0,5397.0,1363.0,871.0,296.0
3220,Puerto Rico,YABUCOAMUNICIPIO,2522.0,3691.0,3370.0,2807.0,2631.0,3991.0,3717.0,3719.0,8584.0,1896.0,1267.0,157.0
